In [5]:
from langchain.chat_models import  ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}.")
prompt = template.format(country_a="France", country_b="Germany")

chat = ChatOpenAI(temperature=0.1)

chat.predict(prompt)

c:\MyWork\PyWork\fullstack_gpt\env\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\MyWork\PyWork\fullstack_gpt\env\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


('There are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [10]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        (
            "human",
            "What is the distance between {country_a} and {country_b}. Also what is your name?",
        ),
    ]
)

prompt = template.format_messages(
    language="Italian", name="Luigi", country_a="France", country_b="Germany"
)
chat.predict_messages(prompt)

NameError: name 'ChatPromptTemplate' is not defined